# ResNet-18 Model From Scratch

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def get_dataloaders(batch_size=256):
    
    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4471), (0.2023, 0.1994, 0.2010))

    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4471), (0.2023, 0.1994, 0.2010))

    ])
    
    train_dataset = CIFAR10(
        root="./data", train=True, download=True, transform=train_transform,
    )

    test_dataset = CIFAR10(
        root="./data", train=False, download=True, transform=test_transform,
    )

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=2
    )

    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=True, num_workers=2
    )

    return train_loader, test_loader

In [ ]:
def get_training_protocol(model, lr=0.1, momentum=0.9, weight_decay=0.0001):
    loss = nn.CrossEntropyLoss()

    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer=optimizer,
        mode='min',
        factor=0.1,
        patience=5,
        verbose=True
    )

    return loss, optimizer, scheduler

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample=None):
        super(BasicBlock, self).__init__()
        
        self.basic_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),

            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1),
            nn.BatchNorm2d(out_channels),
        )

        self.downsample = downsample
    
    def _initial_forward(self, x):
        x = self.basic_block(x)
        return x
    
    def forward(self, x):
        identity = x

        x = self._initial_forwards(x)

        if self.downsample is not None:
            identity = self.downsample(x)
        
        x += identity

        x = F.relu(x)

        return x

In [ ]:
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        pass